In [91]:
import os
import shutil
import numpy as np
from osgeo import gdal, osr

In [92]:
data_dir = r'C:\Personale\atm\reps\gis-programming\osgeopy-data'

### Extracting and saving a subset of an image

In [93]:
# Coordinates for the bounding box to extract.
vashon_ulx, vashon_uly = 532000, 5262600
vashon_lrx, vashon_lry = 548500, 5241500

# Don't forget to change the directory.
os.chdir(os.path.join(data_dir, 'landsat', 'washington'))
in_ds = gdal.Open('nat_color.tif')


In [94]:
# Create an inverse geotransform for the raster. This converts real-world
# coordinates to pixel offsets.
in_gt = in_ds.GetGeoTransform()
inv_gt = gdal.InvGeoTransform(in_gt)
if gdal.VersionInfo()[0] == '1':
    if inv_gt[0] == 1:
        inv_gt = inv_gt[1]
    else:
        raise RuntimeError('Inverse geotransform failed')
elif inv_gt is None:
    raise RuntimeError('Inverse geotransform failed')

In [95]:
# Get the offsets that correspond to the bounding box corner coordinates.
offsets_ul = gdal.ApplyGeoTransform(
    inv_gt, vashon_ulx, vashon_uly)
offsets_lr = gdal.ApplyGeoTransform(
    inv_gt, vashon_lrx, vashon_lry)

In [96]:
# The offsets are returned as floating point, but we need integers.
off_ulx, off_uly = map(int, offsets_ul)
off_lrx, off_lry = map(int, offsets_lr)

In [97]:
# Compute the numbers of rows and columns to extract, based on the offsets.
rows = off_lry - off_uly
columns = off_lrx - off_ulx

In [98]:
# Create an output raster with the correct number of rows and columns.
gtiff_driver = gdal.GetDriverByName('GTiff')
out_ds = gtiff_driver.Create('vashon.tif', columns, rows, 3)
out_ds.SetProjection(in_ds.GetProjection())

0

In [99]:
# Convert the offsets to real-world coordinates for the georeferencing info.
# We can't use the coordinates above because they don't correspond to the
# pixel edges.
subset_ulx, subset_uly = gdal.ApplyGeoTransform(
    in_gt, off_ulx, off_uly)
out_gt = list(in_gt)
out_gt[0] = subset_ulx
out_gt[3] = subset_uly
out_ds.SetGeoTransform(out_gt)

0

In [100]:
# Loop through the 3 bands.
for i in range(1, 4):
    in_band = in_ds.GetRasterBand(i)
    out_band = out_ds.GetRasterBand(i)

    # Read the data from the input raster starting at the computed offsets.
    data = in_band.ReadAsArray(
        off_ulx, off_uly, columns, rows)

    # Write the data to the output, but no offsets are needed because we're
    # filling the entire image.
    out_band.WriteArray(data)

del out_ds

### Ground control points

In [101]:
# Make a copy of the original image so we're leaving it alone and changing
# the new one.

os.chdir(os.path.join(data_dir, 'utah'))
shutil.copy('cache_no_gcp.tif', 'cache.tif')


'cache.tif'

In [102]:
# Open the copied image so we can add GCPs to it.
ds = gdal.Open('cache.tif', gdal.GA_Update)

In [103]:
# Create the SRS that the GCP coordinates use.
sr = osr.SpatialReference()
sr.SetWellKnownGeogCS('WGS84')

0

In [104]:
# Create the list of GCPs.
gcps = [gdal.GCP(-111.931075, 41.745836, 0, 1078, 648),
        gdal.GCP(-111.901655, 41.749269, 0, 3531, 295),
        gdal.GCP(-111.899180, 41.739882, 0, 3722, 1334),
        gdal.GCP(-111.930510, 41.728719, 0, 1102, 2548)]

# Add the GCPs to the raster
ds.SetGCPs(gcps, sr.ExportToWkt())
ds.SetProjection(sr.ExportToWkt())
ds = None

In [105]:
# This time we'll use the driver to make a copy of the raster and then add
# a geotransform instead of GCPs. This still uses the sr and gcps variables
# from above.
old_ds = gdal.Open('cache_no_gcp.tif')
ds = old_ds.GetDriver().CreateCopy('cache2.tif', old_ds)
ds.SetProjection(sr.ExportToWkt())
ds.SetGeoTransform(gdal.GCPsToGeoTransform(gcps))
del ds, old_ds

### Converting pixel coordinates to another image

In [106]:
# Create a function to get the extent of a raster 
def get_extent(fn):
    '''Returns min_x, max_y, max_x, min_y'''
    ds = gdal.Open(fn)
    gt = ds.GetGeoTransform()
    return (gt[0], gt[3], gt[0] + gt[1] * ds.RasterXSize,
        gt[3] + gt[5] * ds.RasterYSize)


In [107]:
# The raster with GCPs doesn't have a geotransform so this extent isn't
# correct.
os.chdir(os.path.join(data_dir, 'utah'))
print(get_extent('cache.tif'))

(0.0, 0.0, 4500.0, 3000.0)


In [108]:
# But this one is.
print(get_extent('cache2.tif'))

(-111.94413990009797, 41.75159324077003, -111.89003929060324, 41.72453336944547)


In [109]:
os.chdir(os.path.join(data_dir, 'landsat', 'washington'))
vashon_ds = gdal.Open('vashon.tif')
full_ds = gdal.Open('nat_color.tif')

In [110]:
# Create a transformer that will map pixel coordinates from the Vashon
# dataset into the full one.
trans = gdal.Transformer(vashon_ds, full_ds, [])

In [111]:
# Use the transformer to figure out the pixel offsets in the full image
# that correspond with the upper left corner of the vashon one.
success, xyz = trans.TransformPoint(False, 0, 0)
print(success, xyz)

1 (6606.0, 3753.0, 0.0)


In [112]:
# If we use the output from that and go the reverse direction, we'll get the
# upper left corner for vashon.
success, xyz = trans.TransformPoint(True, 6606, 3753)
print(success, xyz)

1 (0.0, 0.0, 0.0)


### Color tables

In [113]:
os.chdir(os.path.join(data_dir, 'Switzerland'))
original_ds = gdal.Open('dem_class2.tif')
ds = original_ds.GetDriver().CreateCopy('dem_class3.tif', original_ds)

In [114]:
# Get the existing color table from the band.
band = ds.GetRasterBand(1)
colors = band.GetRasterColorTable()

In [115]:
# Change the entry for 5.
colors.SetColorEntry(5, (250, 250, 250))

In [116]:
# Set the modified color table back on the raster.
band.SetRasterColorTable(colors)
del band, ds

### Transparency

In [117]:
os.chdir(os.path.join(data_dir, 'switzerland'))
original_ds = gdal.Open('dem_class2.tif')
driver = gdal.GetDriverByName('gtiff')

In [118]:
ds = driver.Create('dem_class4.tif', original_ds.RasterXSize,
    original_ds.RasterYSize, 2, gdal.GDT_Byte, ['ALPHA=YES'])

# Add the projection and and geotransform info to the copy.
ds.SetProjection(original_ds.GetProjection())
ds.SetGeoTransform(original_ds.GetGeoTransform())

0

In [119]:
# Read the data in from dem_class2.
original_band1 = original_ds.GetRasterBand(1)
data = original_band1.ReadAsArray()

In [120]:
# Write the data to band 1 of the new raster and copy the color table over.
band1 = ds.GetRasterBand(1)
band1.WriteArray(data)
band1.SetRasterColorTable(original_band1.GetRasterColorTable())
band1.SetRasterColorInterpretation(gdal.GCI_PaletteIndex)
band1.SetNoDataValue(original_band1.GetNoDataValue())

ds.FlushCache()

In [121]:
# Now that we're finally done copying, go back to the text and find
# everywhere in the data array that the pixel value is 5 and set
# the value to 65 instead. Set everything else to 255.
import numpy as np
data = band1.ReadAsArray()
data = np.where(data == 5, 65, 255)

In [122]:
# Now write the modified data array to the second (alpha) band in the new
# raster.
band2 = ds.GetRasterBand(2)
band2.WriteArray(data)
band2.SetRasterColorInterpretation(gdal.GCI_AlphaBand)

del ds, original_ds

### Histograms

In [123]:
# Look at approximate vs exact histogram values.
os.chdir(os.path.join(data_dir, 'switzerland'))
ds = gdal.Open('dem_class2.tif')
band = ds.GetRasterBand(1)
approximate_hist = band.GetHistogram()
exact_hist = band.GetHistogram(approx_ok=False)
print('Approximate:', approximate_hist[:7], sum(approximate_hist))
print('Exact:', exact_hist[:7], sum(exact_hist))

Approximate: [0, 6564, 3441, 3531, 2321, 802, 0] 16659
Exact: [0, 27213, 12986, 13642, 10632, 5414, 0] 69887


In [124]:
# Look at the current default histogram.
print(band.GetDefaultHistogram())

(1.0, 6.0, 3, [40199, 24274, 5414])


In [125]:
# Change the default histogram so that it lumps 1 & 2, 3 & 4, and leaves 5
# by itself.
hist = band.GetHistogram(0.5, 6.5, 3, approx_ok=False)
band.SetDefaultHistogram(1, 6, hist)

0

In [126]:
# Look at what the default histogram is now.
print(band.GetDefaultHistogram())

(1.0, 6.0, 3, [40199, 24274, 5414])


In [127]:
# Get the individual bits of data from the default histogram.
min_val, max_val, n, hist = band.GetDefaultHistogram()
print(min_val, max_val, n)
print(hist)

1.0 6.0 3
[40199, 24274, 5414]


### Reprojecting images

In [128]:
# Reproject the nat_color.tif from UTM to unprojected lat/lon. First create
# the output SRS.
srs = osr.SpatialReference()
srs.SetWellKnownGeogCS('WGS84')

0

In [129]:
# Open the nat_color file.
os.chdir(os.path.join(data_dir, 'landsat', 'washington'))
old_ds = gdal.Open('nat_color.tif')

In [130]:
# Create a VRT in memory that does the reproject.
vrt_ds = gdal.AutoCreateWarpedVRT(old_ds, None, srs.ExportToWkt(),
    gdal.GRA_Bilinear)

In [131]:
# Copy the VRT to a GeoTIFF so we have a file on disk.
gdal.GetDriverByName('gtiff').CreateCopy('nat_color_wgs84.tif', vrt_ds)

<osgeo.gdal.Dataset; proxy of <Swig Object of type 'GDALDatasetShadow *' at 0x000001DF3A571300> >

### Callback functions

In [132]:
# Let's calculate statistics on the natural color Landsat image and show
# progress while it does it (this image probably already has stats, so this
# will go really fast). Watch your output window to see what happens.
os.chdir(os.path.join(data_dir, 'landsat', 'washington'))
ds = gdal.Open('nat_color.tif')
for i in range(ds.RasterCount):
    ds.GetRasterBand(i + 1).ComputeStatistics(False, gdal.TermProgress_nocb)


In [133]:
# How about using the gdal callback function with my own stuff? Let's just
# list all of the files in the current diretory and pretend to do something
# with them.
def process_file(fn):
    # Slow things down a bit by counting to 1,000,000 for each file.
    for i in range(1000000):
        pass # do nothing

list_of_files = os.listdir('.')
for i in range(len(list_of_files)):
    process_file(list_of_files[i])
    gdal.TermProgress_nocb(i / float(len(list_of_files)))
gdal.TermProgress_nocb(100)

1